In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import pandas as pd
from scipy import signal
from dsp import *
from collections import OrderedDict
import warnings
from scipy.io import wavfile
warnings.filterwarnings("ignore")

def make_html_audio(ys, sr, width=100):
    """
    Make an HTML element that contains audio elements in a table
    
    Parameters
    ----------
    ys: list of ndarray
        Audio samples
    sr: int
        Sample rate of audio samples
    """
    clips = []
    for y in ys:
        audio = ipd.Audio(y, rate=sr)
        audio_html = audio._repr_html_().replace('\n', '').strip()
        audio_html = audio_html.replace('<audio ', '<audio style="width: {}px; "'.format(width))
        clips.append(audio_html)
    return clips

def plot_results(S, W, H, win, hop, sr):
    """
    Show the audio components of W and the filtered components
    of S according to a matrix factorixation |S| ~= WH
    
    Parameters
    ----------
    S: ndarray(win, N, dtype=complex)
        Original STFT
    W: ndarray(win, K, dtype=float)
        Sonic template matrix
    H: ndarray(K, N, dtype=float)
        Activations matrix
    win: int
        Window length of STFT
    hop: int
        Hop length of STFT
    sr: int
        Sample rate
    """
    ws = []
    ys = []
    WH = W.dot(H)
    K = H.shape[0]
    for i in range(K):
        # Zero out all activations except for the ith row
        Hi = np.zeros_like(H)
        Hi[i, :] = H[i, :]
        # Pull out just this component
        WHi = np.dot(W, Hi)
        Si = S*WHi/WH # Wiener Filter
        yi = istft(Si, win, hop)
        ys.append(yi)
        
        # Repeat the corresponding column in W a few times
        # to get an idea of what the sonic template is
        Wi = W[:, i]
        Wi = np.reshape(Wi, (Wi.size, 1))
        Wi = Wi*np.ones((1, int(win/hop)))
        wi = gl(Wi, win, hop, blackman_harris_window, 10) 
        ws.append(wi)

    ys = make_html_audio(ys, sr, width=200)
    ws = make_html_audio(ws, sr, width=100)
    pd.set_option('display.max_colwidth', None) 
    df = pd.DataFrame(OrderedDict([("Components", ws), ("Filtered", ys)]))
    return ipd.HTML(df.to_html(escape=False, float_format='%.2f'))

## Euclidean NMF

### Loss Function

### $L(W, H) = \sum_{i, j} (V[i, j]-WH[i, j])^2$
 
### Updates

### $H[i, j] = H[i, j] \frac{ (W^TV)[i, j] }{(W^TWH)[i, j]}$

### $W[i, j] = W[i, j] \frac{ (VH^T)[i, j] }{(WHH^T)[i, j]}$

In [ ]:
def do_nmf_euc(V, K, n_iters):
    """
    Perform Euclidean nonnegative matrix factorization
    
    Parameters
    ----------
    V: ndarray(win, n_frames), nonnegative
        Spectrogram amplitudes to factorize
    K: int
        Number of components to use
    n_iters: int
        Number of iterations to perform
    
    Returns
    -------
    W: ndarray(win, K), nonnegative
        The spectral template columns
    H: ndarray(K, n_frames), nonnegative
        The activations over time
    """
    win = V.shape[0]
    W = np.random.rand(win, K)
    H = np.random.rand(K, V.shape[1])
    ## TODO: Apply multiplicative update rules n_iters times in a loop
    
    return W, H

## Kullback-Liebler Divergence NMF

### Loss function

### $\sum_{i, j} \left( V[i, j] \left( \log \frac{V[i, j]}{WH[i, j]} \right) - V[i, j] + WH[i, j] \right)$

### Updates

### $ V_L[i, j] = V[i, j] / (WH)[i, j] $
### $H[i, j] = H[i, j] \left( \frac{  (W^T V_L)[i, j] }{{\sum_a}W[a, j]} \right) $
### $V_L[i, j] = V[i, j] / (WH)[i, j]$
### $W[i, j] = W[i, j] \left( \frac{  (V_L H^T)[i, j] }{{\sum_b}H[i, b]} \right) $

In [ ]:
def do_nmf_kl(V, K, n_iters):
    """
    Perform Kullback-Liebler nonnegative matrix factorization
    
    Parameters
    ----------
    V: ndarray(win, n_frames), nonnegative
        Spectrogram amplitudes to factorize
    K: int
        Number of components to use
    n_iters: int
        Number of iterations to perform
    
    Returns
    -------
    W: ndarray(win, K), nonnegative
        The spectral template columns
    H: ndarray(K, n_frames), nonnegative
        The activations over time
    """
    win = V.shape[0]
    W = np.random.rand(win, K)
    H = np.random.rand(K, V.shape[1])
    ## TODO: Apply multiplicative update rules n_iters times in a loop
    
    return W, H

In [ ]:
sr, y = wavfile.read("doves.wav")
win = 2048
hop = 512

S = stft(y, win, hop)
V = np.abs(S)

W, H = do_nmf_euc(V, K=3, n_iters=100)

plot_results(S, W, H, win, hop, sr)